# 필요 함수 import

In [1]:
import requests
from bs4 import BeautifulSoup

import re
from collections import Counter
import nltk
from konlpy.tag import Okt

import pandas as pd
import numpy as np

import pickle

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1차 : 쿠팡 상품 리스트 추출
> categoryId
- 밀키트 : 502482
- 1국탕전골 : 502483
- 2덮밥/비빔밥 : 502484
- 3스테이크/고기 502485
- 4면/파스타/감바스 502486
- 5분식 502487
- 6중식요리 502490
- 7기타요리 502491

In [2]:
# 쿠팡 페이지 호출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

## 필요 정보만 추출
- 카테고리 대분류, 상품id, 상품only, 상품명(전체), 정가, 할인율, 판매가, 100g당_가격, 별점, 리뷰수, 품절여부

In [3]:
#카테고리 대분류 라벨링 함수
def category_number(categoryId, category):
    return category.get(categoryId)


#해당 카테고리의 페이지 전체 수 가져오기
def page_number(categoryId, user_agt):
    URL = 'https://www.coupang.com/np/categories/'+categoryId+"?page=1"
    cg_soup_base = get_coupang_item(URL, user_agt)
    page_all = cg_soup_base.select('#product-list-paging > div > a')
    page_list = []
    for page_num in page_all:
        page_num = page_num.text
        try:
            page_num = int(page_num)
            page_list.append(page_num)
        except:
            pass
    return page_list[-1]


#1차 쿠팡 상품 리스트 전처리 함수
def p_list_preprocessing(cg_soup, category_num):
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()
    name_only = name.split(', ')[0]

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text.replace(',','')
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip().replace(',','')
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = np.NaN
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame으로 병합
    coupang_item = pd.DataFrame({'카테고리명':category_num,'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                , '상품':name_only,'상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    return coupang_item

## 쿠팡 상품 리스트 추출 실행부
> 밀키트 추가 필터링 후 전체 크롤링

- 추가 전처리 해야하는 부분
1. 100g당 가격 결측치 처리
2. 구성정보 텍스트 전처리 : 명사 추출 후 재료만 남기기
3. 구성정보 결측치 채우기 : 텍스트로 구성정보가 존재하는 경우만 >> 그외 결측치 삭제
4. 상품명에서 음식명만 추출 > 세부 카테고리 정리

In [4]:
#categoryId 기반 카테고리 대분류 정의
category = {'502483':1, '502484':2, '502485':3, '502486':4, '502487':5, '502490':6, '502491':7}
category_keys = list(category.keys())

#상품 리스트를 담을 빈 dataframe 생성
item_list = ['카테고리명','상품id','data-item-id','data-vendor-item-id','상품','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)

#쿠팡 상품 리스트 추출
cg_url = 'https://www.coupang.com/np/categories/'
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

for categoryId in category_keys:
    page_num = page_number(categoryId, user_agt) #해당 카테고리의 페이지 마지막 페이지 번호 추출
    for page in range(1, page_num+1):
        URL = cg_url+categoryId+"?page="+str(page)+'&filter=1%23attr_12406%2419087%40DEFAULT'
        
        #페이지 호출
        cg_soup_base = get_coupang_item(URL, user_agt)
        cg_soup2 = cg_soup_base.select("#productList li")  

        #상품 리스트 추출 및 전처리
        for cg_soup in cg_soup2:
            coupang_item = p_list_preprocessing(cg_soup, category_number(categoryId, category))
            coupang_items = pd.concat([coupang_items,coupang_item], ignore_index=True)

coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                373 non-null    object
 1   상품id                 373 non-null    object
 2   data-item-id         373 non-null    object
 3   data-vendor-item-id  373 non-null    object
 4   상품                   373 non-null    object
 5   상품명                  373 non-null    object
 6   정가                   373 non-null    object
 7   할인율                  373 non-null    object
 8   판매가                  373 non-null    object
 9   100g당_가격             330 non-null    object
 10  별점                   373 non-null    object
 11  리뷰수                  373 non-null    object
 12  품절여부                 373 non-null    object
dtypes: object(13)
memory usage: 38.0+ KB


In [5]:
#전체 중복행 확인
coupang_items.loc[coupang_items.duplicated(subset=['상품id']), :]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부


In [6]:
#중복행 삭제
coupang_items.drop_duplicates('상품id', inplace=True)
coupang_items.reset_index(drop=True, inplace=True)

# 2차 : 구성 정보 추출
- 상품id, 구성정보

In [7]:
#2차 : 구성정보 추출 함수
def p_c_extraction(productId, p_c_page):
    product_composition = '0'
    for item in p_c_page:
        s = item.text.replace('\n','').strip()
        if product_composition == '2': #3: 재료부분만 추출
            product_composition = s
        elif product_composition =='1':#2: 불필요한 text 제외
            product_composition='2'
        elif '구성 정보' in s :         #1: '구성 정보' 존재여부 확인
            product_composition = '1'

    #4: 없으면 결측값 처리
    if product_composition.isdigit():
        product_composition = np.NaN
    
    data_item = pd.DataFrame({'상품id':productId, '구성정보':product_composition}, index=[0])
    return data_item

## 2차 실행부

In [8]:
#2차 실행부
data_items = pd.DataFrame(columns=['상품id', '구성정보'])

for idx in range(len(coupang_items)):
    #각 상품 상세페이지 호출
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.loc[idx, '상품id'] #data-product-id
    itemsId = coupang_items.loc[idx, 'data-item-id']  #data-item-id
    vendorItemId = coupang_items.loc[idx, 'data-vendor-item-id'] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    pdt_soup = get_coupang_item(URL, user_agt)
    
    #'구성정보' 추출
    p_c_page = pdt_soup.find('body').children
    data_item = p_c_extraction(productId, p_c_page)
    data_items = pd.concat([data_items, data_item], ignore_index=True)

data_items.head()

,상품id,구성정보
0,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
1,293421715,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채..."
2,4926044090,NaN
3,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성..."
4,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다."


# 상세페이지 이미지에서 구성정보 추출


In [9]:
def get_coupang_image(img_URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    page = requests.get(img_URL, headers= headers).text
    #coupang_image_soup = BeautifulSoup(page.text, "html.parser")
    #return coupang_image_soup
    return page

In [10]:
# 구성정보 페이지에서 전체 이미지 링크 추출 -> 리스트화
image_link = []
product_url = 'https://www.coupang.com/vp/products/'
image_productId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['상품id'].values)
image_dataitemId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['data-item-id'].values)
image_datavendorId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['data-vendor-item-id'].values)

for product in range(len(image_productId)):
    img_URL = product_url + str(image_productId[product]) + '/items/' +str(image_dataitemId[product])+'/vendoritems/'+str(image_datavendorId[product])
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    page = get_coupang_image(img_URL, user_agt)

    link_start = []
    for text in re.finditer('//thumbnail',page):
        link_start.append(text.start())
        
    link_end = []
    for text in re.finditer('g"}],',page):
        link_end.append(text.start())
    
    for num in range(len(link_start)):
        image_link.append('https:' + page[int(link_start[num]):int(link_end[num]) + 1])
        

In [35]:
## 상세페이지에서 전체 이미지 저장
import urllib.request

count = 0
for url in image_link:
    count += 1
    path = "C:/coding/multicampus/image/" + (str(count) + ".jpg")
    urllib.request.urlretrieve(url, path)


In [36]:
# 이미지에서 텍스트 추출 코드 with pytesserect
import pytesseract
import cv2 
import matplotlib.pyplot as plt
from PIL import Image

extracted_text = []
for num in range(1, count + 1):
    file = 'C:/coding/multicampus/image/' + str(num) + '.jpg'
    image = Image.open(file)
    if image.size[1] >= 4000:
        croppedImage = image.crop((0,2000,image.size[0],3350))
        croppedImage.save('C:/coding/multicampus/image/cropped' + str(num) + '.jpg')
        file = 'C:/coding/multicampus/image/cropped' + str(num) + '.jpg'
        image = cv2.imread(file)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        text = pytesseract.image_to_string(rgb_image, lang='kor')
        extracted_text.append(text)
    else:
        image = cv2.imread(file)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        text = pytesseract.image_to_string(rgb_image, lang='kor')
        extracted_text.append(text)

In [37]:
# 결측값 중 곰곰 밀키트 구성정보만 테서렉트로 이미지 텍스트 추출 후,
# 오타 교정 위해 전처리 작업
gomgom_text = [i for i in extracted_text if "구성되어 있어요"  in i] 
gomgom = []
for i in gomgom_text:
    sentence = re.sub('(<([^>]+)>)', '', i)
    sentence = re.sub('[^ ㄱ-ㅎㅏ-ㅣ각-힣]', ' ', sentence)
    gomgom.append(sentence)
    
gom_typo_list = [] 

for gom in gomgom:
    gg = gom.split(' ')
    gom_typo = []
    for i in range(len(gg)):
        if len(gg[i]) > 1:
            gom_typo.append(gg[i])
        else:
            pass
    gom_typo_list.append(gom_typo)

In [38]:
# gom_type_list 피클화
import pickle

with open('gom_typo.pkl','wb') as f:
    pickle.dump(gom_typo_list,f)

# 곰곰 구성재료 오타 교정 (Colab)

In [ ]:
# !pip install symspellpy jamo hangul_utils
# !wget https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ko/ko_50k.txt
# from symspellpy import SymSpell, Verbosity
# from symspellpy_ko import KoSymSpell, Verbosity
# from hangul_utils import split_syllable_char, split_syllables, join_jamos
# from jamo import h2j, j2hcj
# import pandas as pd
# import unicodedata

# def to_jamos(text):
#   return j2hcj(h2j(text))


# 만개의 레시피 재료 목록 자소 분리
# vocab = pd.read_csv('만개의레시피_재료_목록_오타처리용.txt', sep = " ", names = ["term", "count"])
# vocab.head()
# vocab.info()
# vocab = vocab.dropna(axis=0)

# vocab.term = vocab.term.map(to_jamos)
# vocab.to_csv("만개의레시피_재료_목록_오타처리용_decomposed.txt", sep=" ", header=None, index=None)
# vocab.head()

In [ ]:
# # 저장했던 리스트 불러옴
# with open('gom_typo.pkl','rb') as f:
#   gom = pickle.load(f)
  
  
# for g in gom:
#     idx = g.index('있어요')
#     del g[0:idx + 1]

# for g in gom:
#   if '일은' in g :
#     idx = g.index('일은')
#     del g[idx :]

#   elif 'ㅇㅇ' in g :
#     idx = g.index('ㅇㅇ')
#     del g[idx :]
  
#   else:
#     pass

# sym_spell = SymSpell(max_dictionary_edit_distance = 3)
# dictionary_path = '만개의레시피_재료_목록_오타처리용_decomposed.txt'
# sym_spell.load_dictionary(dictionary_path, 0, 1)

# # gom_typo_fix에 오타 교정된 재료 목록 저장
# gom_typo_fix = []
# for g in gom:
#   g_typo_fix = []
#   for term in g:
#     term = split_syllables(term) 
#     suggestions = sym_spell.lookup(term, Verbosity.ALL, max_edit_distance = 3)
#     for sugg in suggestions :
#       if sugg.distance < 2:
#         typofix = join_jamos(sugg.term)
#         g_typo_fix.append(typofix)
#         break
#       else:
#         pass
#   gom_typo_fix.append(g_typo_fix)
  
  
# gom_typo_fix를 다시 피클로 저장
# with open('gom_typo_fix.pkl','wb') as f:
#     pickle.dump(gom_typo_fix,f)


## 결측치에 곰곰 채우기

In [39]:
#곰곰 상품id 추출
gomgom_productId = list(coupang_items.loc[coupang_items['상품'].str.contains('곰곰')]['상품id'].values)

#곰곰 이미지에서 추출한 구성정보 list
with open('../../data/gom_typo_fix.pkl', 'rb') as fr:
    gomgom_data = pickle.load(fr)

#상품id + 구성정보
gomgom_df = pd.DataFrame(zip(gomgom_productId, gomgom_data), columns=['상품id', '구성정보'])

### 곰곰 결측치 채우기
for i in range(len(gomgom_df)):
    idx = data_items[data_items['상품id'] == gomgom_productId[i]].index.values
    gomgom = gomgom_df.loc[i,'구성정보']
    data_items.loc[idx, '구성정보'] = ', '.join(gomgom)
data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    373 non-null    object
 1   구성정보    330 non-null    object
dtypes: object(2)
memory usage: 6.0+ KB


## 구성정보 전처리 후 재대입

In [40]:
#구성정보 재료 전처리
def p_c_preprocessing(product_composition):
    product_composition = re.sub(r'\\n', '', product_composition)
    product_composition = re.sub(r'\)', ',', product_composition)
    product_composition = re.sub(r'[^ㄱ-ㅣ가-힣a-zA-Z\,\(\:]', ' ', product_composition)
    product_composition = product_composition.strip()
    
    replace_list = ['으로 구성되어 있습니다','으로 구성되어있습니다','로 구성되어 있습니다','로 구성되어 있어요','로 구성되어있습니다']
    for r in replace_list:
        product_composition = product_composition.replace(r, '')
    
    replace_list = ['과', '와']
    for r in replace_list:
        product_composition = product_composition.replace(r, ',') 
    
    # 추가 작업에 따라 추가 여부 결정
    # if ('야채' in product_composition) or ('채소(' in product_composition) or ('채소 :' in product_composition):
    #     replace_list = ['야채 세트', '혼합 채소', '혼합 해물', '채소(', '채소 :']
    #     for r in replace_list:
    #         product_composition = re.sub('[\(\)\:]', ',', product_composition)
    #         product_composition = product_composition.replace(r, '')
    # else:
    product_composition = re.sub('[\(\)\:]', ',', product_composition)

    return product_composition

In [41]:
#요리재료내용 결측치 삭제
data_items_drop = data_items[data_items.loc[:,'구성정보'].isnull()].index
data_items_drop = data_items.drop(data_items_drop)
data_items_drop.reset_index(drop=True, inplace=True)
data_items_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    330 non-null    object
 1   구성정보    330 non-null    object
dtypes: object(2)
memory usage: 5.3+ KB


>> 소고기 == 쇠고기 이건 어디에..?

In [42]:
#요리재료_전처리(new column) 
data_items_drop['구성정보_전처리'] = np.NaN

for idx in range(len(data_items_drop)):
    ingredient = data_items_drop.loc[idx, '구성정보']
    ingredients = p_c_preprocessing(ingredient)
    ingredients = ingredients.split(',')
    
    ingre_lists = []
    for i in ingredients:
        i = i.strip()
        replace_list = ['칼국수 면', '다진 마늘', '대구 곤이', '명태 곤이']
        if i in replace_list:
            i = i.split(' ')
            i = ''.join(i)

        i = i.split(' ')
        if i[-1] == '등' :  # 등을 그대로 두는 것이 나을지 지금처럼 빼는게 나을지..?
            i.pop(-1)
            i = ' '.join(i)
            ingre_lists.append(i)
        else:
            i = ' '.join(i)
            ingre_lists.append(i)
        
    #null값 제거
    ingre_lists = list(filter(None, ingre_lists))

    #요리재료_전처리(new column)에 추가
    data_items_drop.loc[idx, '구성정보_전처리'] = ', '.join(ingre_lists)

data_items_drop.iloc[0:10, :]

,상품id,구성정보,구성정보_전처리
0,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육, 소곱창, 소대창, 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
1,293421715,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기, 만두, 생면, 소스 종, 육수, 표고버섯, 팽이버섯, 채소, 배추, 청경..."
2,4926044090,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면..."
3,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스"
4,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물, 채소, 연두부, 다시팩, 소스"
5,6310053892,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기, 콩나물, 대파, 양파, 깻잎, 소스"
6,6764028903,"기타 수산물가공품(명태알, 대구곤이, 새우), 채소(무, 대파, 청양고추), 소스로...","기타 수산물가공품, 명태알, 대구곤이, 새우, 채소, 무, 대파, 청양고추, 소스"
7,1637283332,"순두부, 애호박, 소고기, 팽이버섯, 대파, 소스","순두부, 애호박, 소고기, 팽이버섯, 대파, 소스"
8,1946869683,"애호박, 양파, 대파, 청양고추, 두부, 팽이버섯","애호박, 양파, 대파, 청양고추, 두부, 팽이버섯"
9,188130036,"소고기, 당면, 팽이버섯, 소스, 참기름 등으로 구성되어 있습니다.","소고기, 당면, 팽이버섯, 소스, 참기름"


# 비교를 위한 리스트 생성

In [43]:
# 만개의 레시피 재료 목록 read
ingre_10000 = []
with open('./만개의레시피_재료_목록.txt', 'r') as f :
    lines = f.readlines()
    lines = str(lines).split(',')
    for line in lines:
        ingre_10000.append(line) 
len(ingre_10000)

37970

In [44]:
#쿠팡 재료 목록 생성
ingre_coupang_list = data_items_drop['구성정보_전처리'].dropna()
ingre_coupang_lists = ingre_coupang_list.to_list()

ingre_coupangs = []
for item in ingre_coupang_lists:
    ingre_coupang=[]
    items = item.split(', ')
    for i in items:
        ingre_coupang.append(i)
    ingre_coupangs.append(ingre_coupang)

# #쿠팡 재료 목록 저장
# with open('쿠팡_재료_목록.txt', 'w') as f :
#     for ingre in ingre_coupang:
#         f.write(ingre)

In [ ]:
#쿠팡에만 있는 재료만
ingre_not_10000 = []
for items in ingre_coupangs :
    not_10000 = []
    for i in items:
        if not i in ingre_10000 and i != '':
            not_10000.append(i)
    ingre_not_10000.append(not_10000)
print(len(ingre_not_10000))
ingre_not_10000

In [ ]:
data_items_drop['구성정보_전처리']

In [45]:
#일치하는 재료 리스트로 만들고 데이터 프레임으로 수정
ingre_10000s = []
for items in ingre_coupangs :
    ingre_10000_list = []
    for i in items:
        if i in ingre_10000:
            ingre_10000_list.append(i)
    ingre_10000s.append(ingre_10000_list)
print(len(ingre_10000s))
ingre_10000s

330


[['양념육',
  '소곱창',
  '소대창',
  '전골용 육수',
  '새송이버섯',
  '느타리버섯',
  '소고기',
  '두부',
  '배추',
  '양파',
  '깻잎',
  '라면사리'],
 ['소고기', '만두', '생면', '육수', '표고버섯', '팽이버섯', '채소', '배추', '청경채', '깻잎'],
 ['쇠고기',
  '새송이버섯',
  '표고버섯',
  '초간장소스',
  '칠리소스',
  '샤브용육수',
  '생면',
  '칼국수면',
  '숙면',
  '감자수제비',
  '깻잎',
  '팽이버섯',
  '배추',
  '청경채'],
 ['소시지', '오뗄팜S', '프레스햄', '소고기', '치즈', '라면사리', '채소', '김치', '소스'],
 ['해물', '채소', '연두부', '다시팩', '소스'],
 ['소고기', '콩나물', '대파', '양파', '깻잎', '소스'],
 ['명태알', '대구곤이', '새우', '채소', '무', '대파', '청양고추', '소스'],
 ['순두부', '애호박', '소고기', '팽이버섯', '대파', '소스'],
 ['애호박', '양파', '대파', '청양고추', '두부', '팽이버섯'],
 ['소고기', '당면', '팽이버섯', '소스', '참기름'],
 ['두부', '청국장', '애호박', '돼지고기', '대파', '양파', '혼합장'],
 ['돼지고기', '야채', '애호박', '감자', '느타리버섯', '양파', '청양고추', '대파', '소스'],
 ['돼지고기', '모듬햄', '대파', '양파', '양배추', '라면사리', '부대찌개소스', '김치', '슬라이스', '치즈'],
 ['소 스지', '전각 슬라이스'],
 ['소고기', '소스', '쥬키니호박', '팽이버섯', '대파', '청양고추', '두부'],
 ['왕만두', '소스', '칼국수면', '느타리버섯', '대파'],
 ['홍합', '문어', '오징어', '꽃게', '명태알', '대구곤이', '새우', '해물탕 양념', 

In [46]:
for idx in range(len(ingre_10000s)):
    data_items_drop.loc[idx, '구성정보_전처리'] = ','.join(ingre_10000s[idx])
data_items_drop.head()

,상품id,구성정보,구성정보_전처리
0,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎..."
1,293421715,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎"
2,4926044090,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수..."
3,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스"
4,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스"


## 긴 텍스트에서 사전과 일치하는 요리재료만 추출

In [ ]:
#요리재료 사전과 일치여부를 통해 요리재료 추출
# ingredients_list = []
# for coupangs in ingre_coupangs :
#     ingre_list = []
#     for coupang in coupangs:
#         if coupang in ingre_10000:
#             ingre_list.append(coupang)
#         else:
#             ingre = [ man for man in ingre_10000 if man in coupang]
#             ingre_list.extend(ingre)
#     ingredients_list.append(ingre_list)
#     print(ingre_list)
#ingredients_list

# 쿠팡 상품 리스트와 구성정보 병합

In [47]:
#쿠팡 상품 리스트와 구성정보 병합
coupang = pd.merge(coupang_items, data_items_drop)
coupang.to_csv('coupang_filtering.csv')
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                330 non-null    object
 1   상품id                 330 non-null    object
 2   data-item-id         330 non-null    object
 3   data-vendor-item-id  330 non-null    object
 4   상품                   330 non-null    object
 5   상품명                  330 non-null    object
 6   정가                   330 non-null    object
 7   할인율                  330 non-null    object
 8   판매가                  330 non-null    object
 9   100g당_가격             289 non-null    object
 10  별점                   330 non-null    object
 11  리뷰수                  330 non-null    object
 12  품절여부                 330 non-null    object
 13  구성정보                 330 non-null    object
 14  구성정보_전처리             330 non-null    object
dtypes: object(15)
memory usage: 41.2+ KB


In [21]:
coupang.head()

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,7%,24210,"(100g당 1,519원)",4.5,11262,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎..."
1,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.5,49201,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎"
2,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",20810,28%,14900,"(100g당 1,717원)",5.0,36322,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스"
3,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12051,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스"
4,1,6310053892,13092768757,80353833768,마이셰프 EBS 최요비 산더미 소고기 콩불 2인분,"마이셰프 EBS 최요비 산더미 소고기 콩불 2인분, 810g, 1개",14900,34%,9800,"(100g당 1,210원)",4.5,2656,0,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기,콩나물,대파,양파,깻잎,소스"


In [48]:
#구성정보 결측치 확인
print('결측치 개수: ',len(coupang.loc[coupang['구성정보_전처리']=='']))
coupang.loc[coupang['구성정보_전처리']=='']

결측치 개수:  8


,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
85,1,2140731697,3635578248,71621124717,설래담 민어 매운탕 (냉동),"설래담 민어 매운탕 (냉동), 680g, 1개",15190,0,15190,"(100g당 2,234원)",4.0,335,0,"신선한 민어, 풍성한 야채 믹스, 얼큰한 풍미의 매운탕 소스, 시원한 맛을 더해줄 ...",
139,3,5965059546,10701364550,77982083557,마이셰프 명절 밀키트 궁중 소고기 갈비찜 1026g + 소고기버섯잡채 571g + ...,마이셰프 명절 밀키트 궁중 소고기 갈비찜 1026g + 소고기버섯잡채 571g + ...,66700,11%,58900,NaN,5.0,24,0,"궁중 소고기 갈비찜, 일품 소고기 육전, 소고기 버섯 잡채로 구성되어 있습니다.",
140,3,5965059575,10701364629,77982083715,마이셰프 명절 밀키트 버섯가득 소불고기전골 905g + 소고기버섯잡채 571g + ...,마이셰프 명절 밀키트 버섯가득 소불고기전골 905g + 소고기버섯잡채 571g + ...,43700,15%,36900,NaN,4.5,21,0,"버섯가득 소불고기 전골, 궁중 소불고기, 소고기 버섯 잡채로 구성되어 있습니다.",
235,7,6703653767,15536769131,82755955008,프렙 이탈리안 감바스 알 아히요 (냉동),"프렙 이탈리안 감바스 알 아히요 (냉동), 300g, 1개",13200,0,13200,"(100g당 4,400원)",4.5,43,0,포장 정보,
249,7,6058464775,11130341505,78408934862,복선당 국내산 제주고기 돼지수육 김치찜 850g (냉동),"복선당 국내산 제주고기 돼지수육 김치찜 850g (냉동), 1세트",15900,0,15900,NaN,4.5,837,0,김치찜과 돼지수육으로 구성되어 있습니다.,
276,7,6811645649,16116095590,83313001742,집반찬연구소 간편 짜장용 손질야채 240g x 2개 + 청정원 직화 짜장 80g 세트,집반찬연구소 간편 짜장용 손질야채 240g x 2개 + 청정원 직화 짜장 80g 세...,7900,0,7900,NaN,5.0,5,0,청정원 직화짜장과 손질 야채로 구성되어 있습니다.,
316,7,6703653501,15536767789,82755953691,프렙 볼로네제 리조또 (냉동),"프렙 볼로네제 리조또 (냉동), 520g, 1개",11200,0,11200,"(100g당 2,154원)",5.0,37,1,포장 정보,
322,7,6703653639,15536768551,82755954355,프렙 버터 전복 리조또 (냉동),"프렙 버터 전복 리조또 (냉동), 427g, 1개",11600,0,11600,"(100g당 2,717원)",4.5,58,1,포장 정보,


In [49]:
coupang.loc[coupang['구성정보_전처리']==''].index

Int64Index([85, 139, 140, 235, 249, 276, 316, 322], dtype='int64')

In [50]:
#우선 결측치 제거
coupang_null_list = coupang.loc[coupang['구성정보_전처리']==''].index
coupang.drop(coupang_null_list, inplace=True)
coupang.loc[coupang['구성정보_전처리']=='']

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리


In [51]:
coupang.to_csv('coupang_filtering.csv')
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 0 to 329
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                322 non-null    object
 1   상품id                 322 non-null    object
 2   data-item-id         322 non-null    object
 3   data-vendor-item-id  322 non-null    object
 4   상품                   322 non-null    object
 5   상품명                  322 non-null    object
 6   정가                   322 non-null    object
 7   할인율                  322 non-null    object
 8   판매가                  322 non-null    object
 9   100g당_가격             285 non-null    object
 10  별점                   322 non-null    object
 11  리뷰수                  322 non-null    object
 12  품절여부                 322 non-null    object
 13  구성정보                 322 non-null    object
 14  구성정보_전처리             322 non-null    object
dtypes: object(15)
memory usage: 40.2+ KB
